# <center> <img src="../../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Spring 2025** </center>
---
### <center> **Kafka Producer: Financial Transaction Generator** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

**Team members**: 
- Miguel Alberto Torres Dueñas
- Juan Pablo Cortez Navarro
- Luther Williams Sandria 
- Ferdinand Bierbaum

In [20]:
import findspark
findspark.init()
#0be7b65b50a239d7ee8b621f3c329b25c5c4aadafbae5ac7

In [21]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Kafka") \
    .master("spark://f5db43ce3d38:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
sc = spark.sparkContext

In [22]:
kafka_lines = spark \
                .readStream \
                .format("kafka") \
                .option("kafka.bootstrap.servers", "67261d1016d7:9093") \
                .option("subscribe", "kafka-spark-example-0") \
                .option("subscribe", "kafka-spark-example-1") \
                .option("subscribe", "kafka-spark-example-2") \
                .option("subscribe", "kafka-spark-example-3") \
                .load()

kafka_lines.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [ ]:
from pyspark.sql.functions import split, col, expr

kafka_df = kafka_lines.select(split(col("value"), ",").alias("pairs_array"))

kafka_df = kafka_df.withColumn("user_id", split(col("pairs_array").getItem(0), ":").getItem(1))
kafka_df = kafka_df.withColumn("video_id", split(col("pairs_array").getItem(1), ":").getItem(1))
kafka_df = kafka_df.withColumn("watch_time_seconds", split(col("pairs_array").getItem(2), ":").getItem(1))
kafka_df = kafka_df.withColumn("resolution", split(col("pairs_array").getItem(3), ":").getItem(1))
kafka_df = kafka_df.withColumn("bitrate_kbps", split(col("pairs_array").getItem(4), ":").getItem(1))
kafka_df = kafka_df.withColumn("buffering_events", split(col("pairs_array").getItem(5), ":").getItem(1))
kafka_df = kafka_df.withColumn("paused", split(col("pairs_array").getItem(6), ":").getItem(1))
kafka_df = kafka_df.withColumn("skipped", split(col("pairs_array").getItem(7), ":").getItem(1))
kafka_df = kafka_df.withColumn("genre", split(col("pairs_array").getItem(8), ":").getItem(1))
kafka_df = kafka_df.withColumn("region", split(col("pairs_array").getItem(9), ":").getItem(1))
kafka_df = kafka_df.withColumn("recommended", split(col("pairs_array").getItem(10), ":").getItem(1))

# Usamos expr para hacer la resta de longitud directamente
kafka_df = kafka_df.withColumn(
    "timestamp",
    expr("substring(split(pairs_array[11], ':')[1], 1, length(split(pairs_array[11], ':')[1]) - 1)")
)

kafka_df.printSchema()


root
 |-- pairs_array: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- user_id: string (nullable = true)
 |-- video_id: string (nullable = true)
 |-- watch_time_seconds: string (nullable = true)
 |-- resolution: string (nullable = true)
 |-- bitrate_kbps: string (nullable = true)
 |-- buffering_events: string (nullable = true)
 |-- paused: string (nullable = true)
 |-- skipped: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- region: string (nullable = true)
 |-- recommended: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [ ]:
# query = kafka_df \
#                 .writeStream \
#                 .outputMode("append") \
#                 .trigger(processingTime='3 seconds') \
#                 .format("console") \
#                 .option("truncate", "false") \
#                 .start()

# query.awaitTermination(300)
# query.stop()

In [28]:
query_files = kafka_df \
                .writeStream \
                .outputMode("append") \
                .trigger(processingTime='3 seconds') \
                .format("parquet") \
                .option("path", "/home/jovyan/notebooks/data/project_parquet") \
                .option("truncate", "false") \
                .option("checkpointLocation", "/home/jovyan/checkpoint") \
                .start()
query_files.awaitTermination(300)
query_files.stop()

25/05/12 00:13:34 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/12 00:13:35 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/05/12 00:16:24 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 3000 milliseconds, but spent 3005 milliseconds
25/05/12 00:16:36 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 3000 milliseconds, but spent 3280 milliseconds
25/05/12 00:16:39 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 3000 milliseconds, but spent 3092 milliseconds
25/05/12 00:16:48 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 3000 milliseconds, but spent 3641 milliseconds
25/05/12 00:17:00 WARN ProcessingTimeExecutor: Current 

In [29]:
df = spark.read.parquet("/home/jovyan/notebooks/data/project_parquet")
df.show(5, False)


+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+----------+------------------+----------+------------+----------------+------+-------+--------------+------+-----------+-------------+
|pairs_array                                                                                                                                                                                                                                                                           |user_id     |video_id  |watch_time_seconds|resolution|bitrate_kbps|buffering_events|paused|skipped|genre         |region|recommended|timestamp    |
+-----------------------------------------------------------------------------------------------------------------------------------------------

IllegalArgumentException: Data type array<string> of column pairs_array is not supported.
Data type string of column user_id is not supported.
Data type string of column video_id is not supported.
Data type string of column watch_time_seconds is not supported.
Data type string of column resolution is not supported.
Data type string of column bitrate_kbps is not supported.
Data type string of column buffering_events is not supported.
Data type string of column paused is not supported.
Data type string of column skipped is not supported.
Data type string of column genre is not supported.
Data type string of column region is not supported.
Data type string of column recommended is not supported.

In [ ]:
sc.stop()